In [18]:
import requests
from bs4 import BeautifulSoup
from pylatexenc.latex2text import LatexNodes2Text
import re

BASE_URL = "https://m.loigiaihay.com"
START_URL = "https://m.loigiaihay.com/bai-3-thu-tu-trong-tap-hop-cac-so-tu-nhien-e22312.html"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def get_exercise_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = []
    # Lấy link bài tập từ ul.list_new.list_cont li h3.magb5 > a
    for li in soup.select('ul.list_new.list_cont li'):
        a = li.select_one('h3.magb5 > a')
        if a:
            href = a.get('href')
            if href and href.startswith('/'):
                links.append(BASE_URL + href)
            elif href and href.startswith('http'):
                links.append(href)
    return links

def extract_qa_from_exercise(html):
    soup = BeautifulSoup(html, 'html.parser')
    box = soup.find('div', id='box-content')
    question = ''
    answer = ''
    if box:
        # Tìm <strong class="content_question">Đề bài</strong> và lấy tất cả <p> liên tiếp sau đó, dừng nếu gặp <strong class="content_detail">
        q_strong = box.find('strong', class_='content_question')
        if q_strong:
            p = q_strong.find_parent('p')
            if p:
                question_parts = []
                sib = p.find_next_sibling('p')
                while sib and sib.name == 'p':
                    # Nếu trong <p> này có <strong class="content_detail"> thì dừng lại
                    strong = sib.find('strong', class_='content_detail')
                    if strong:
                        break
                    txt = sib.get_text(strip=True)
                    if txt:
                        question_parts.append(txt)
                    sib = sib.find_next_sibling('p')
                question = '\n'.join(question_parts)
        # Tìm <strong class="content_detail">Lời giải chi tiết</strong> và lấy các <p> tiếp theo
        a_strong = box.find('strong', class_='content_detail')
        if a_strong:
            p = a_strong.find_parent('p')
            if p:
                answer_parts = []
                sib = p.find_next_sibling('p')
                while sib and sib.name == 'p':
                    txt = sib.get_text(strip=True)
                    if txt:
                        answer_parts.append(txt)
                    sib = sib.find_next_sibling('p')
                answer = '\n'.join(answer_parts)
    return question, answer

def latex_to_unicode(text):
    # Tìm tất cả biểu thức LaTeX trong \( ... \) hoặc \[ ... \] và chuyển sang unicode
    def repl(m):
        expr = m.group(1)
        return LatexNodes2Text().latex_to_text(expr)
    # \( ... \)
    text = re.sub(r'\\\((.+?)\\\)', repl, text)
    text = re.sub(r'\\\[(.+?)\\\]', repl, text)
    text = re.sub(r'\((.+?)\)', repl, text)
    text = re.sub(r'\[(.+?)\]', repl, text)
    # Loại bỏ các ký tự escape \\ (dòng mới)
    text = text.replace('\\', '')
    return text

resp = requests.get(START_URL, headers=headers)
links = get_exercise_links(resp.text)
results = []
for url in links:
    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        q, a = extract_qa_from_exercise(r.text)
        if q and a:
            results.append({'url': url, 'question': q, 'answer': a})
    except Exception as e:
        print(f"Error with {url}: {e}")
for item in results:
    q = latex_to_unicode(item['question'])
    a = latex_to_unicode(item['answer'])
    print(f"URL: {item['url']}")
    print(f"Question: {q}\n")
    print(f"Answer: {a}\n")
    print('-'*40)

URL: https://m.loigiaihay.com/tra-loi-hoat-dong-1-trang-13-sgk-toan-6-ket-noi-tri-thuc-voi-cuoc-song-a92269.html
Question: Trong hai điểm 5 và 8 trên tia số, điểm nào nằm trên trái, điểm nào nằm bên phải điểm kia?

Answer: Điểm 5 nằm bên trái điểm 8, điểm 8 nằm bên phải điểm 5.

----------------------------------------
URL: https://m.loigiaihay.com/tra-loi-hoat-dong-2-trang-13-sgk-toan-6-ket-noi-tri-thuc-voi-cuoc-song-a92270.html
Question: Điểm biểu diễn số tự nhiên nào nằm ngay bên trái điểm 8?
Điểm biểu diễn số tự nhiên nào nằm ngay bên phải điểm 8?

Answer: Điểm 7 biểu diễn số tự nhiên nằm ngay bên trái điểm 8.
Điểm 9 biểu diễn số tự nhiên nằm ngay bên phải điểm 8.

----------------------------------------
URL: https://m.loigiaihay.com/tra-loi-hoat-dong-3-trang-13-sgk-toan-6-ket-noi-tri-thuc-voi-cuoc-song-a92271.html
Question: Cho n là một số tự nhiên nhỏ hơn 7. Theo em, điểm n nằm bên trái hay bên phải điểm 7?

Answer: n là một số tự nhiên nhỏ hơn 7 thì điểm n nằm bên trái điểm 7.


In [ ]:
# Save to a JSON file
with open("math_exercises.json", "w", encoding="utf-8") as f:
    json.dump(unique_exercises, f, ensure_ascii=False, indent=4)
print("All exercises saved to math_exercises.json")

In [20]:
url = 'https://m.loigiaihay.com/toan-lop-6-ket-noi-tri-thuc-voi-cuoc-song-c643.html'
bs4 = requests.get(url)

soup = BeautifulSoup(bs4.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="vi">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Toán 6, giải toán lớp 6 kết nối tri thức với cuộc sống</title>
<meta content="" name="keywords"/>
<meta content="Giải sgk toán 6 tập 1, tập 2 kết nối tri thức giúp học sinh soạn toán 6, giải toán đại số và hình học 6 hay nhất, đầy đủ lý thuyết, bài tập trắc nghiệm" name="description"/>
<meta content="index,follow" name="robots"/><meta content="Loigiaihay.com" property="og:site_name"/>
<meta content="1188256281282988" property="fb:app_id"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<meta content="https://img.loigiaihay.com/picture/article/2021/1011/sgk-kntt-643.png" property="og:image"/>
<link href="https://img.loigiaihay.com/picture/article/2021/1011/sgk-kntt-643.png" rel="image_src"/>
<link href="https://loigiaihay.com/toan-lop-6-ket-noi-tri-thuc-voi-cuoc-song-c643.html" rel="canonical"/>
<link href="https://loigiaihay.com/to

In [22]:
import requests
from bs4 import BeautifulSoup
from pylatexenc.latex2text import LatexNodes2Text
import re

BASE_URL = "https://m.loigiaihay.com"
MAIN_URL = "https://m.loigiaihay.com/toan-lop-6-ket-noi-tri-thuc-voi-cuoc-song-c643.html"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def get_all_lesson_urls_from_url(url):
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    urls = []
    # Tìm tất cả các thẻ <a> có href chứa '/bai-' hoặc 'luyen-tap' hoặc 'cuoi-chuong' (bài, luyện tập, bài tập cuối chương)
    for a in soup.find_all('a', href=True):
        href = a['href']
        if href.startswith('http') and (
            '/bai-' in href or 'luyen-tap' in href or 'cuoi-chuong' in href):
            urls.append(href)
    return urls

def get_exercise_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = []
    # Lấy link bài tập từ ul.list_new.list_cont li h3.magb5 > a
    for li in soup.select('ul.list_new.list_cont li'):
        a = li.select_one('h3.magb5 > a')
        if a:
            href = a.get('href')
            if href and href.startswith('/'):
                links.append(BASE_URL + href)
            elif href and href.startswith('http'):
                links.append(href)
    return links

def extract_qa_from_exercise(html):
    soup = BeautifulSoup(html, 'html.parser')
    box = soup.find('div', id='box-content')
    question = ''
    answer = ''
    if box:
        # Tìm <strong class="content_question">Đề bài</strong> và lấy tất cả <p> liên tiếp sau đó, dừng nếu gặp <strong class="content_detail">
        q_strong = box.find('strong', class_='content_question')
        if q_strong:
            p = q_strong.find_parent('p')
            if p:
                question_parts = []
                sib = p.find_next_sibling('p')
                while sib and sib.name == 'p':
                    # Nếu trong <p> này có <strong class="content_detail"> thì dừng lại
                    strong = sib.find('strong', class_='content_detail')
                    if strong:
                        break
                    txt = sib.get_text(strip=True)
                    if txt:
                        question_parts.append(txt)
                    sib = sib.find_next_sibling('p')
                question = '\n'.join(question_parts)
        # Tìm <strong class="content_detail">Lời giải chi tiết</strong> và lấy các <p> tiếp theo
        a_strong = box.find('strong', class_='content_detail')
        if a_strong:
            p = a_strong.find_parent('p')
            if p:
                answer_parts = []
                sib = p.find_next_sibling('p')
                while sib and sib.name == 'p':
                    txt = sib.get_text(strip=True)
                    if txt:
                        answer_parts.append(txt)
                    sib = sib.find_next_sibling('p')
                answer = '\n'.join(answer_parts)
    return question, answer

def latex_to_unicode(text):
    # Tìm tất cả biểu thức LaTeX trong \( ... \) hoặc \[ ... \] và chuyển sang unicode
    def repl(m):
        expr = m.group(1)
        return LatexNodes2Text().latex_to_text(expr)
    # \( ... \)
    text = re.sub(r'\\\((.+?)\\\)', repl, text)
    text = re.sub(r'\\\[(.+?)\\\]', repl, text)
    text = re.sub(r'\((.+?)\)', repl, text)
    text = re.sub(r'\[(.+?)\]', repl, text)
    # Loại bỏ các ký tự escape \\ (dòng mới)
    text = text.replace('\\', '')
    return text

# Lấy danh sách URL bài toán từ link tổng
lesson_urls = get_all_lesson_urls_from_url(MAIN_URL)
results = []
for START_URL in lesson_urls:
    try:
        resp = requests.get(START_URL, headers=headers)
        links = get_exercise_links(resp.text)
        for url in links:
            try:
                r = requests.get(url, headers=headers)
                r.raise_for_status()
                q, a = extract_qa_from_exercise(r.text)
                if q and a:
                    results.append({'url': url, 'question': q, 'answer': a})
            except Exception as e:
                print(f"Error with {url}: {e}")
    except Exception as e:
        print(f"Error with {START_URL}: {e}")
for item in results:
    q = latex_to_unicode(item['question'])
    a = latex_to_unicode(item['answer'])
    print(f"URL: {item['url']}")
    print(f"Question: {q}\n")
    print(f"Answer: {a}\n")
    print('-'*40)

URL: https://m.loigiaihay.com/tra-loi-luyen-tap-1-trang-6-sgk-toan-6-ket-noi-tri-thuc-voi-cuoc-song-a92245.html
Question: Gọi B là tập hợp các bạn tổ trưởng trong lớp em. Em hãy nêu tên một bạn thuộc tập B và một bạn không thuộc tập B.

Answer: Tất cả các bạn tổ trưởng trong lớp em đều thuộc B.
Ví dụ: An, Minh, Tâm, Huy, Khang, Ngọc
Một bạn thuộc B: Em tìm một bạn tổ trưởng.
Ví dụ: Tâm
Một bạn không thuộc B: bạn không là tổ trưởng.
Ví dụ: Linh Chi
Loigiaihay.com

----------------------------------------
URL: https://m.loigiaihay.com/tra-loi-cau-hoi-trang-7-sgk-toan-6-ket-noi-tri-thuc-voi-cuoc-song-a92246.html
Question: Khi mô tả tập hợp L các chữ cái trong từ NHATRANG bằng cách liệt kê các phần tử, bạn Nam viết:
L = {N, H, A, T, R A, N, G}.
Theo em, bạn Nam viết đúng hay sai?

Answer: Bạn Nam viết sai vì mỗi phần tử chỉ được viết một lần mà phần tử A và N bạn Nam viết 2 lần.
Cách viết đúng: L = {N;H;A;T;R;G}

----------------------------------------
URL: https://m.loigiaihay.com/tra-lo